In [ ]:
#@title Installing necessary libraries, that are not already installed on Colab
# This might take a couple of minutes, and it may restart the runtime!
# If the runtime requests reloading, canceling is usually the better.
# (This might change in the future.)

# Detectron2 environment for instance segmentation
!pip install "git+https://github.com/facebookresearch/detectron2.git"

# SMP environment for tracking
!pip install segmentation-models-pytorch

In [5]:
#@title Importing required functions and libraries

# Imports from Symmetry-Tracker repo

import sys
sys.path.append("/path/to/symmetry_tracker/")

from symmetry_tracker.general_functionalities.video_transformation import TransformVideoFromTIFF

from symmetry_tracker.segmentation.segmentator import SingleVideoSegmentation
from symmetry_tracker.segmentation.segmentation_io import DisplaySegmentation, WriteSegmentation

from symmetry_tracker.tracking.tracker import SingleVideoObjectTracking
from symmetry_tracker.tracking.tracking_io import DisplayTracks, WriteTracks, SaveTracksVideo
from symmetry_tracker.tracking.post_processing import InterpolateMissingObjects

# Other necessary imports

import torch

In [ ]:
#@title Downloading the Models and Sample Data

import os

!mkdir downloads

segmentator_models_dir = "/path/to/models/"
segmentator_model_name = "model_final.pth"
segmentator_config_name = "config.yaml"

# Tracking model
tracking_models_dir = "/path/to/models/"
tracking_model_name = "MOTSynth_KernelTracker_[DLV3p,resnet50]_FBtr2_Ep4_Adv2_final.pth"

# Sample recording
sample_record_name = "600"
sample_records_dir = f"/path/to/transformed/data/recordings/frames/{sample_record_name}/rgb/"

!wget --no-clobber $segmentator_models_dir$segmentator_model_name -P downloads/
!wget --no-clobber $segmentator_models_dir$segmentator_config_name -P downloads/
!wget --no-clobber $tracking_models_dir$tracking_model_name -P downloads/

!wget -r -nd -np -A jpg,jpeg,png $sample_records_dir -P downloads/sample_images/$sample_record_name/


In [ ]:
#@title Pipeline parameter setup

# Input paths
SegmentationModelPath = "./downloads/"+segmentator_model_name
SegmentationModelConfigPath = "./downloads/"+segmentator_config_name
TrackingModelPath = "./downloads/"+tracking_model_name

!mkdir inputs
!mkdir inputs/frames
TransformedVideoPath = "./inputs/frames/"

# Output paths
!mkdir outputs
!mkdir outputs/segmentations
!mkdir outputs/trackings
!mkdir outputs/videos
SegmentationSavePath = "./outputs/segmentations/"+sample_record_name+"_Segmentation.txt"
TrackingSavePath = "./outputs/trackings/"+sample_record_name+"_Tracks.json"
TrackingWritePath = "./outputs/trackings/"+sample_record_name+"_Tracks.txt"
TrackingVideoPath = "./outputs/videos/"+sample_record_name+"_Tracks.mp4"

# TimeKernelSize is the size of the kernel in both directions without the central image
# (So if TimeKernelSize is 2, then the full kernel size is 5)
# This parameter is unique to the given tracking model
TimeKernelSize = 2

# Matching colab environment (for now GPU vs CPU)
Device = ("cuda:0" if torch.cuda.is_available() else "cpu")
print("Colab environment: "+Device)

In [6]:
#@title Video Transformation

import shutil

max_sample_num = None

shutil.rmtree(TransformedVideoPath)
os.mkdir(TransformedVideoPath)

all_sample_names = sorted(os.listdir(f"downloads/sample_images/{sample_record_name}/"))
for sample_pos in range(len(all_sample_names)):
  if max_sample_num is None or sample_pos < max_sample_num:
    shutil.copy(os.path.join(f"downloads/sample_images/{sample_record_name}/", all_sample_names[sample_pos]),
                TransformedVideoPath)


In [6]:
#@title Segmentation Modifications

import numpy as np
import cv2
import os
from pycocotools import mask as coco_mask
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor

try:
  from IPython.display import display
  from symmetry_tracker.general_functionalities.misc_utilities import progress
except:
  pass

def PerformSegmentation(Predictor, VideoPath, Color = "GRAYSCALE", MinObjectSize = None):
  VideoFrames = sorted(os.listdir(VideoPath))
  Outmasks = {}

  NumFrames = len(VideoFrames)
  print("Performing segmentation")
  try:
    ProgressBar = display(progress(0, NumFrames), display_id=True)
  except:
    pass
  for Frame in range(0,NumFrames):
    try:
      ProgressBar.update(progress(Frame, NumFrames))
    except:
      pass
    if Color == "GRAYSCALE":
      Img = cv2.imread(os.path.join(VideoPath,VideoFrames[Frame]), cv2.IMREAD_GRAYSCALE)
      Img = np.expand_dims(Img, axis=2)
    elif Color == "RGB":
      Img = cv2.cvtColor(cv2.imread(os.path.join(VideoPath, VideoFrames[Frame])), cv2.COLOR_BGR2RGB)
    else:
      raise Exception(f"{Color} is an invalid keyword for Color")

    Outputs = Predictor(Img)
    Outmask = (Outputs["instances"].pred_masks.to("cpu").numpy())
    ReducedOutmask = []
    for Segment in Outmask:
      if MinObjectSize is None or np.sum(Segment) >= MinObjectSize:
        ReducedOutmask.append(coco_mask.encode(np.asfortranarray(Segment)))
    Outmasks[Frame] = ReducedOutmask
  try:
    ProgressBar.update(progress(1, 1))
  except:
    pass
  print("Segmentation finished")
  return Outmasks

def SingleVideoSegmentation(VideoPath, ModelPath, ModelConfigPath, Device, Color = "GRAYSCALE", ScoreThreshold = 0.2, MinObjectSize = None):
  """
  - VideoPath: The path to the video (in .png images format) to be segmented
  - ModelPath: The path to the model description file (.pth)
  - ModelConfigPath: The path to the model configuration file (.yaml)
  - Device: The device on which the segmentator should run (cpu or cuda:0)
  - Color: A keyword specific to the used model on the color encoding
           Available options: GRAYSCALE and RGB
  - ScoreThreshold: The acceptance threshold defining the object-ness of a given pixel
                    Lower values are more accepting
  - MinObjectSize: An optional parameter defining the minimal required object size in terms of pixel area
                  None is the default value
  """
  if not Color in ["GRAYSCALE", "RGB"]:
    raise Exception(f"{Color} is an invalid keyword for Color")

  cfg = get_cfg()
  cfg.merge_from_file(ModelConfigPath)
  cfg.MODEL.DEVICE = Device
  cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST=ScoreThreshold
  cfg.MODEL.WEIGHTS = ModelPath
  Predictor = DefaultPredictor(cfg)
  Outmasks = PerformSegmentation(Predictor, VideoPath, Color, MinObjectSize)
  return Outmasks

In [7]:
#@title Segmentation_io Modifications

import numpy as np
import cv2
import os
from pycocotools import mask as coco_mask
import matplotlib.pyplot as plt

def DisplaySegmentation(VideoPath, Outmasks, DisplayFrameNumber = True, Figsize=(4,4)):
  """
  Displays the segmentation in Outmasks onto the video at VideoPath

  - VideoPath: The video to be displayed in standard .png images format
  - Outmasks: The segmentation results coming from SingleVideoSegmentation()
  - DisplayFrameNumber: Boolean variable marking whether to display the frame number
  - Figsize: The shape of the figure in standard matplotlib format
  """
  VideoFrames = sorted(os.listdir(VideoPath))
  for Frame in range(len(VideoFrames)):
    fig, (ax1) = plt.subplots(1, 1, figsize=Figsize)
    VideoFrame = cv2.cvtColor(cv2.imread(os.path.join(VideoPath, VideoFrames[Frame])), cv2.COLOR_BGR2RGB)
    ax1.imshow(VideoFrame, interpolation='nearest')
    if Frame in Outmasks.keys():
      SegmentsSum = np.zeros(np.shape(VideoFrame)[0:2])
      for segment in Outmasks[Frame]:
        SegmentsSum += coco_mask.decode(segment)
      ax1.imshow(SegmentsSum>0, cmap=plt.cm.hot, vmax=2, alpha=.3, interpolation='bilinear')
      if DisplayFrameNumber:
        ax1.text(3, 20, "Frame "+str(Frame+1), color="deepskyblue")
    plt.show()

def WriteSegmentation(Outmasks, SavePath):
  """
  Saves the segmentation to a txt file

  - Outmasks: The segmentation results coming from SingleVideoSegmentation()
  - SavePath: The path to the file, where the segmentation will be saved
                          If the file does not exists, it will be created
  """
  with open(SavePath, "w") as OutFile:
    OutFile.write("POS\tF\tCELLNUM\tX\tY\n")
    for Frame in Outmasks:
      CellCount=1
      for segment in Outmasks[Frame]:
        contours, hierarchy  = cv2.findContours(np.array(coco_mask.decode(segment), dtype = np.uint8) ,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        for contour in contours:
          for a in contour:
            l=a[0]
            OutFile.write("1"+"\t"+str(Frame+1)+"\t"+"{:04d}".format(Frame+1)+"{:04d}".format(CellCount)+"\t"+str(l[0])+"\t"+str(l[1])+"\n")
          CellCount+=1
  print("Segmentation saved to: "+SavePath)

In [8]:
#@title Instance Segmentation

# Performing segmentation
Outmasks = SingleVideoSegmentation(TransformedVideoPath, SegmentationModelPath, SegmentationModelConfigPath, Device, Color = "RGB", ScoreThreshold = 0.4)

# Displaying segmentation results
#DisplaySegmentation(TransformedVideoPath, Outmasks)

# Saving segmentation
WriteSegmentation(Outmasks, SegmentationSavePath)

Performing segmentation


/usr/local/lib/python3.10/dist-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Segmentation finished
Segmentation saved to: ./outputs/segmentations/600_Segmentation.txt


In [9]:
#@title Tracker Metrics Modifications

import numpy as np

def TracksIOU(Array1, Array2, dt):
  if Array1.shape != Array2.shape:
    raise Exception(f"Dimensions of Array1 {Array1.shape} and Array2 {Array2.shape} must be the same")

  Intersection = np.count_nonzero(np.logical_and(Array1[dt:], Array2[:-dt]))
  Union = np.count_nonzero(np.logical_or(Array1[dt:], Array2[:-dt]))

  if Union == 0:
    return 0
  try:
    return Intersection / Union
  except:
    return 0

def SegmentationIOU(Bin1, Bin2):
  if Bin1.shape != Bin2.shape:
    raise Exception(f"Dimensions of Bin1 {Bin1.shape} and Bin2 {Bin2.shape} must be the same")

  Intersection = np.count_nonzero(np.logical_and(Bin1,Bin2))
  Union = np.count_nonzero(np.logical_or(Bin1,Bin2))

  if Union == 0:
    return 0
  try:
    return Intersection / Union
  except:
    return 0

def AnnotationOverlap(Annot1, Annot2):
  if Annot1.shape != Annot2.shape:
    raise Exception(f"Dimensions of Annot1 {Annot1.shape} and Annot2 {Annot2.shape} must be the same")

  Intersection = np.sum(Annot1 * Annot2)
  if Intersection == 0:
    return 0

  SmallerArea = min([np.sum(Annot1), np.sum(Annot2)])

  if SmallerArea == 0:
    return 0
  try:
    return Intersection / SmallerArea
  except:
    return 0

In [10]:
#@title Tracker Utilities Modifications

import numpy as np
import cv2
import torch
import pandas as pd
from pycocotools import mask as coco_mask

from symmetry_tracker.general_functionalities.misc_utilities import CenterMass, BoxOverlap, BoundingBox
#from symmetry_tracker.tracking.tracker_metrics import AnnotationOverlap

try:
  from IPython.display import display
  from symmetry_tracker.general_functionalities.misc_utilities import progress
except:
  pass


def RemoveFaultyObjects(AnnotDF, VideoShape, MinObjectPixelNumber, MaxOverlapRatio):
  """
  Removes the objects from the annotation, which
  - have too small size based on self.MinObjectPixelNumber
  - have no defineable center
  """
  print("Removing faulty object instances")

  Counter = 0
  FaultyInstances = {}

  NumFrames = len(AnnotDF["Frame"].unique())
  try:
    ProgressBar = display(progress(0, NumFrames), display_id=True)
  except:
    pass

  for Frame in AnnotDF["Frame"].unique():

    try:
      ProgressBar.update(progress(Frame, NumFrames))
    except:
      pass

    FaultyInstances[Frame]=[]
    ObjectIDs = AnnotDF.query("Frame == @Frame")["ObjectID"]

    for ObjectID in ObjectIDs:
      ObjectBbox = AnnotDF.query("ObjectID == @ObjectID")["SegBbox"].iloc[0]
      ObjectSeg = coco_mask.decode(AnnotDF.query("ObjectID == @ObjectID")["SegmentationRLE"].iloc[0])
      Size = np.sum(ObjectSeg)
      [Cx,Cy] = AnnotDF.query("ObjectID == @ObjectID")["Centroid"].iloc[0]

      HasBetterCoverage = False
      for Object2ID in ObjectIDs:
        if ObjectID != Object2ID:
          Object2Bbox = AnnotDF.query("ObjectID == @Object2ID")["SegBbox"].iloc[0]

          if BoxOverlap(ObjectBbox, Object2Bbox) > 0:
            Object2Seg = coco_mask.decode(AnnotDF.query("ObjectID == @Object2ID")["SegmentationRLE"].iloc[0])
            if np.sum(ObjectSeg) <= np.sum(Object2Seg) and AnnotationOverlap(ObjectSeg, Object2Seg) > MaxOverlapRatio:
              HasBetterCoverage = True

      if Size<MinObjectPixelNumber or [Cx, Cy]==[None,None] or Cx<=0 or Cy<=0 or Cx>=VideoShape[2]-1 or Cy>=VideoShape[1]-1 or HasBetterCoverage:
        FaultyInstances[Frame].append(ObjectID)
        Counter+=1

  try:
    ProgressBar.update(progress(1, 1))
  except:
    pass

  for Frame in FaultyInstances:
    for ObjectID in FaultyInstances[Frame]:
      AnnotDF = AnnotDF.query("ObjectID != @ObjectID")

  print(f"Number of removed faulty object instances: {Counter}")
  return AnnotDF

#ZeroFrame can be 0 or 1 (depending on where the counting starts)
def LoadAnnotationDF(AnnotPath, VideoShape, MinObjectPixelNumber = 20, MaxOverlapRatio = 0.2, ZeroFrame = 1, FaultyObjectRemoval = True):
  print("Loading Annotation from:")
  print(AnnotPath)
  AnnotFile = open(AnnotPath, 'r')
  AnnotDF = pd.DataFrame(columns = ["Frame", "ObjectID", "SegmentationRLE", "LocalTrackRLE",
                                    "Centroid", "SegBbox", "TrackBbox", "PrevID", "NextID", "TrackID", "Interpolated"])
  FirstRow = True
  PrevObjectID = -1
  PrevFrame = -1
  PolyLine = []
  NewObjectID = 1
  for line in AnnotFile:
    if not FirstRow and line:
      splitted = line.split()
      Frame = int(splitted[1])
      ObjectID = splitted[2]
      x = int(float(splitted[3]))
      y = int(float(splitted[4]))
      if (ObjectID != PrevObjectID and PrevObjectID!=-1) or (Frame != PrevFrame and PrevFrame != -1):
        IndividualSegImg = np.zeros([VideoShape[1],VideoShape[2]])
        cv2.fillPoly(IndividualSegImg,np.int32([PolyLine]),1)
        IndividualSegImg = np.array(IndividualSegImg, dtype=bool)
        Centroid = CenterMass(IndividualSegImg)
        Bbox = BoundingBox(IndividualSegImg)
        FullObjectID = "{:04d}".format(PrevFrame - ZeroFrame)+"{:04d}".format(NewObjectID)
        IndividualSegImgRLE = coco_mask.encode(np.asfortranarray(IndividualSegImg))
        AnnotRow = pd.Series({"Frame": PrevFrame - ZeroFrame, "ObjectID": FullObjectID, "SegmentationRLE": IndividualSegImgRLE, "LocalTrackRLE": None,
                      "Centroid": Centroid, "SegBbox":Bbox, "TrackBbox": None, "PrevID": None, "NextID": None, "TrackID": None, "Interpolated": False})
        AnnotDF = pd.concat([AnnotDF, AnnotRow.to_frame().T], ignore_index=True)
        PolyLine = []
        NewObjectID += 1
        if Frame != PrevFrame:
          NewObjectID = 1
      PrevObjectID = ObjectID
      PrevFrame = Frame
      PolyLine.append([x,y])
    FirstRow = False

  if FaultyObjectRemoval:
    AnnotDF = RemoveFaultyObjects(AnnotDF, VideoShape, MinObjectPixelNumber, MaxOverlapRatio)

  return AnnotDF

def LoadPretrainedModel(ModelPath, Device):
  Model = torch.load(ModelPath, map_location = torch.device(Device))
  Model.eval()
  print("Model successfully loaded from:")
  print(ModelPath)
  return Model

In [11]:
#@title Tracker Modifications

import time
import cv2
import os
import numpy as np
import torch
import torch.nn.functional as F
import gc
from scipy.optimize import linear_sum_assignment

from symmetry_tracker.general_functionalities.misc_utilities import EncodeMultiRLE, DecodeMultiRLE, OuterBoundingBox, BoxOverlap, dfs
#from symmetry_tracker.tracking.tracker_metrics import TracksIOU
#from symmetry_tracker.tracking.tracker_utilities import LoadAnnotationDF, LoadPretrainedModel

try:
  from IPython.display import display
  from symmetry_tracker.general_functionalities.misc_utilities import progress
except:
  pass


def KernelTrackCentroid(LocalVideo, VideoShape, Model, Device, SegmentationConfidence, ObjectCenter):
  with torch.no_grad():
    inputs = LocalVideo
    CPImage = np.zeros([VideoShape[1],VideoShape[2]])
    [CMy,CMx]=ObjectCenter
    CPImage[CMx-2:CMx+2,CMy-2:CMy+2]=255
    inputs = np.append(inputs, [CPImage], axis=0)
    inputs = np.array(inputs, dtype=float)/255
    inputs = torch.Tensor(np.array(inputs))

    pad_h = (16 - inputs.shape[1] % 16) % 16
    pad_w = (16 - inputs.shape[2] % 16) % 16
    inputs = F.pad(inputs, (0, pad_w, 0, pad_h), mode='constant', value=0)

    inputs=torch.unsqueeze(inputs, dim=0)

    inputs=inputs.to(torch.device(Device))
    output = np.array(torch.sigmoid(Model(inputs).cpu()))
    output = output>SegmentationConfidence
    output = output*1.0
    output = np.nan_to_num(output, nan=0.0, posinf=1.0, neginf=0.0)

    torch.cuda.empty_cache()

    """
    plt.imshow(output[0, 0, :, :])
    plt.show()
    """
  return np.array(output[0], dtype = bool)

def KernelTrackBbox(LocalVideo, VideoShape, Model, Device, SegmentationConfidence, ObjectBbox):
  with torch.no_grad():
    inputs = LocalVideo
    BboxImg = np.zeros([VideoShape[1],VideoShape[2]])
    [x0, y0, x1, y1] = ObjectBbox
    BboxImg[x0:x1,y0:y1]=255
    inputs = np.append(inputs, [BboxImg], axis=0)
    inputs = np.array(inputs, dtype=float)/255
    inputs = torch.Tensor(np.array(inputs))

    pad_h = (16 - inputs.shape[1] % 16) % 16
    pad_w = (16 - inputs.shape[2] % 16) % 16
    inputs = F.pad(inputs, (0, pad_w, 0, pad_h), mode='constant', value=0)

    inputs=torch.unsqueeze(inputs, dim=0)

    inputs=inputs.to(torch.device(Device))
    output = np.array(torch.sigmoid(Model(inputs).cpu()))
    output = output>SegmentationConfidence
    output = output*1.0
    output = np.nan_to_num(output, nan=0.0, posinf=1.0, neginf=0.0)

    torch.cuda.empty_cache()

    """
    plt.imshow(output[0, 0, :, :])
    plt.show()
    """
  return np.array(output[0], dtype = bool)


def LocalTracking(VideoPath, VideoShape, AnnotDF, Model, Device, TimeKernelSize, Color = "GRAYSCALE", Marker = "CENTROID", SegmentationConfidence = 0.2):

  if not Color in ["GRAYSCALE", "RGB"]:
    raise Exception(f"{Color} is an invalid keyword for Color")
  if not Marker in ["CENTROID", "BBOX"]:
    raise Exception(f"{Marker} is not an appropriate keyword for Marker")

  VideoFrames = sorted(os.listdir(VideoPath))
  NumFrames = len(VideoFrames)

  print("Local Tracking")
  try:
    ProgressBar = display(progress(0, NumFrames), display_id=True)
  except:
    pass

  for Frame in range(NumFrames):
    ObjectIDs = AnnotDF.query("Frame == @Frame")["ObjectID"]
    for ObjectID in ObjectIDs:

      # Input image Composition

      if Color == "GRAYSCALE":
        CentralImg = cv2.imread(os.path.join(VideoPath,VideoFrames[Frame]), cv2.IMREAD_GRAYSCALE)
        LocalVideo = np.repeat(CentralImg[np.newaxis, ...], 2*TimeKernelSize+1, axis=0)
        for dt in range(-TimeKernelSize, TimeKernelSize+1):
          if Frame+dt >= 0 and Frame+dt < NumFrames and dt != 0:
            LocalVideo[dt+TimeKernelSize] = cv2.imread(os.path.join(VideoPath,VideoFrames[Frame+dt]), cv2.IMREAD_GRAYSCALE)

      elif Color == "RGB":
        CentralImg = cv2.cvtColor(cv2.imread(os.path.join(VideoPath, VideoFrames[Frame])), cv2.COLOR_BGR2RGB)
        CentralImg = np.transpose(CentralImg, (2,0,1))
        NumReps = 2*TimeKernelSize+1
        LocalVideo = np.zeros((3*NumReps,
                       np.shape(CentralImg)[1],
                       np.shape(CentralImg)[2]),
                      dtype=CentralImg.dtype)
        for Rep in range(NumReps):
          LocalVideo[3*Rep:3*Rep+3] = CentralImg
        for dt in range(-TimeKernelSize, TimeKernelSize+1):
          if Frame+dt >= 0 and Frame+dt < NumFrames and dt != 0:
            LocalImg = cv2.cvtColor(cv2.imread(os.path.join(VideoPath, VideoFrames[Frame+dt])), cv2.COLOR_BGR2RGB)
            LocalVideo[3*(dt+TimeKernelSize):3*(dt+TimeKernelSize)+3] = np.transpose(LocalImg, (2,0,1))

      else:
        raise Exception(f"{Color} is an invalid keyword for Color")

      # Local Tracking

      LocalTrack = None

      if Marker == "CENTROID":
        ObjectCenter = AnnotDF.query("ObjectID == @ObjectID")["Centroid"].iloc[0]
        LocalTrack = KernelTrackCentroid(LocalVideo, VideoShape, Model, Device, SegmentationConfidence, ObjectCenter)

      elif Marker == "BBOX":
        ObjectBbox = AnnotDF.query("ObjectID == @ObjectID")["SegBbox"].iloc[0]
        LocalTrack = KernelTrackBbox(LocalVideo, VideoShape, Model, Device, SegmentationConfidence, ObjectBbox)

      AnnotDF.loc[AnnotDF.query("ObjectID == @ObjectID").index, "LocalTrackRLE"] = [EncodeMultiRLE(LocalTrack)]

      # 3D Boundary Box calculation

      bbox = OuterBoundingBox(LocalTrack)
      AnnotDF.loc[AnnotDF.query("ObjectID == @ObjectID").index, "TrackBbox"] = [bbox]

    try:
      ProgressBar.update(progress(Frame, NumFrames))
    except:
      pass

  try:
    ProgressBar.update(progress(1, 1))
  except:
    pass

  return AnnotDF


def GlobalAssignment(VideoPath, VideoShape, AnnotDF, TimeKernelSize, MinRequiredSimilarity=0.5, MaxTimeKernelShift=None):

  VideoFrames = sorted(os.listdir(VideoPath))
  NumFrames = len(VideoFrames)

  if MaxTimeKernelShift is None:
    MaxTimeKernelShift=TimeKernelSize*2

  print("Global Assignment")

  try:
    ProgressBar = display(progress(0, len(AnnotDF)), display_id=True)
  except:
    pass

  AnnotDF[["PrevID", "NextID"]] = None
  for dt in range(1,MaxTimeKernelShift):
    for Frame in range(NumFrames-dt):

      # Similarity Matrix Calculation

      t0 = time.time()

      T0_IDs = AnnotDF.query("Frame == @Frame and NextID.isnull()", engine='python')["ObjectID"].tolist()
      Tdt_IDs = AnnotDF.query("Frame == @Frame+@dt and PrevID.isnull()", engine='python')["ObjectID"].tolist()

      SimilarityMatrix = np.zeros((len(T0_IDs),len(Tdt_IDs)))

      for i in range(len(T0_IDs)):
        T0_ID = T0_IDs[i]

        LTR0 = DecodeMultiRLE(AnnotDF.query("ObjectID == @T0_ID")["LocalTrackRLE"].iloc[0])
        for j in range(len(Tdt_IDs)):
          Tdt_ID = Tdt_IDs[j]

          bbox0 = AnnotDF.query("ObjectID == @T0_ID")["TrackBbox"].iloc[0]
          bboxdt = AnnotDF.query("ObjectID == @Tdt_ID")["TrackBbox"].iloc[0]
          t00 = time.time()
          overlap = BoxOverlap(bbox0, bboxdt)
          if overlap != 0:
            LTRdt = DecodeMultiRLE(AnnotDF.query("ObjectID == @Tdt_ID")["LocalTrackRLE"].iloc[0])
            IOU = TracksIOU(LTR0, LTRdt, dt)
            SimilarityMatrix[i, j] = IOU

      SimilarityMatrix = SimilarityMatrix * (SimilarityMatrix >= MinRequiredSimilarity)

      """
      sns.heatmap(SimilarityMatrix)
      plt.show()
      """

      # Hungarian Method based Assignment

      try:
        T0_assignedVals, Tdt_assignedVals = linear_sum_assignment(1-SimilarityMatrix)
      except:
        print(f"Error in linear_sum_assignment at Frame {Frame} to Frame {Frame+dt}")
        continue

      for k in range(len(T0_assignedVals)):
        if SimilarityMatrix[T0_assignedVals[k], Tdt_assignedVals[k]] >= MinRequiredSimilarity:
          T0_ID = T0_IDs[T0_assignedVals[k]]
          Tdt_ID = Tdt_IDs[Tdt_assignedVals[k]]
          AnnotDF.loc[AnnotDF.query("ObjectID == @T0_ID").index, "NextID"] = Tdt_ID
          AnnotDF.loc[AnnotDF.query("ObjectID == @Tdt_ID").index, "PrevID"] = T0_ID

      t_end = time.time()

      """
      print(f"dt {dt}, Frame {Frame}, t_full {t_end-t0} s")
      """

      try:
        ProgressBar.update(progress(len(AnnotDF.query("not NextID.isnull()")), len(AnnotDF)))
      except:
        pass

  try:
    ProgressBar.update(progress(1, 1))
  except:
    pass

  return AnnotDF


def ConnectedIDReduction(AnnotDF):

  AnnotDF["TrackID"] = None

  # Transforming key-value pair matches to an undirected graph dictionary

  EquivalencyGraph = {}
  for ObjectID in AnnotDF["ObjectID"].unique():
    Neighbors = set()
    PrevID = AnnotDF.query("ObjectID == @ObjectID")["PrevID"].iloc[0]
    NextID = AnnotDF.query("ObjectID == @ObjectID")["NextID"].iloc[0]
    if not PrevID is None:
      Neighbors.add(PrevID)
    if not NextID is None:
      Neighbors.add(NextID)
    EquivalencyGraph[ObjectID] = Neighbors

  # Creating equivalency sets (everything that is connected is equivavlent)

  EquivalencySets = []
  for RootCandidate in EquivalencyGraph.keys():
    CandidateInEqSets=False
    for EqSet in EquivalencySets:
      if RootCandidate in EqSet:
        CandidateInEqSets=True
    if not CandidateInEqSets:
      Visited=set()
      dfs(Visited, EquivalencyGraph, RootCandidate)
      EquivalencySets.append(Visited)

  # Generating new (minimal) IDs into the AnnotDF

  NewTrackID = 1
  for EquivalentIDs in EquivalencySets:
    for ObjectID in EquivalentIDs:
      AnnotDF.loc[AnnotDF.query("ObjectID == @ObjectID").index, "TrackID"] = NewTrackID

    NewTrackID += 1

  return AnnotDF


def SingleVideoObjectTracking(VideoPath, ModelPath, Device, AnnotPath, TimeKernelSize,
                              Color = "GRAYSCALE", Marker = "CENTROID", MinObjectPixelNumber=20, SegmentationConfidence = 0.1,
                              MinRequiredSimilarity=0.5, MaxOverlapRatio=0.5, MaxTimeKernelShift=None):
  """
  - VideoPath: The path to video in stardard .png images format on which the tracking will be performed
  - ModelPath: The path to the pretrained model (the full model definition, not just the state dictionary)
  - Device: The device on which the segmentator should run (cpu or cuda:0)
  - AnnotPath: The path to a single annotation (segmentation) belonging to the video at VideoPath
  - TimeKernelSize: A constant parameter for the trained Tracker.
                    TimeKernelSize is the "radius" of the kernel, TimeKernelSize*2+1 is the "diamater" of the actual kernel.
  - Color: A keyword specific to the used model on the color encoding
           Available options: GRAYSCALE and RGB
  - Marker: A keyword specific to the used model on how the object to be tracked is marked
            Available options: CENTROID and BBOX
  - MinObjectPixelNumber: Defines the minimal number of pixels in a Object istance for the instance to be recognised as valid
                        Object instances with PixelNumber<MinObjectPixelNumber will be simply deleted during initiation
  - SegmentationConfidence: A number in [0,1] or defining the confidence threshold for the segmentation
                            Lower values are more allowing. Recommanded values are in the [0.1,0.9] range
  - MinRequiredSimilarity: The minimal required similarity based on IOU for two trackings to be possibly counted as belonging to the same Object
  - MaxOverlapRatio:  The maximal overlap allowed between annotations in the original annotation.
                      Above MaxOverlapRatio, the area-wise smaller Object will be removed.
                      Not an important parameter if the segmentation is more or less a partitioning
  - MaxTimeKernelShift: The maximal shift allowed between trackings to be recognised as belonging to the same Object
                        Minimal possible value: 1
                        Maximal possible value: 2*TimeKernelSize
                        The default None means maximal possible value
                        Usually None is recommended
                        Smaller values may result in trackings with more "breaks", but possibly fewer errors and slightly faster calculation
  """

  if not Color in ["GRAYSCALE", "RGB"]:
    raise Exception(f"{Color} is an invalid keyword for Color")
  if not Marker in ["CENTROID", "BBOX"]:
    raise Exception(f"{Marker} is not an appropriate keyword for Marker")

  VideoFrames = sorted(os.listdir(VideoPath))
  Img0 = cv2.imread(os.path.join(VideoPath,VideoFrames[0]))
  VideoShape = [len(os.listdir(VideoPath)), np.shape(Img0)[0], np.shape(Img0)[1]]
  AnnotDF = LoadAnnotationDF(AnnotPath, VideoShape, MinObjectPixelNumber, MaxOverlapRatio)

  Model = LoadPretrainedModel(ModelPath, Device)
  AnnotDF = LocalTracking(VideoPath, VideoShape, AnnotDF, Model, Device, TimeKernelSize, Color, Marker, SegmentationConfidence)
  del Model
  gc.collect()

  AnnotDF = GlobalAssignment(VideoPath, VideoShape, AnnotDF, TimeKernelSize, MinRequiredSimilarity, MaxTimeKernelShift)

  AnnotDF = ConnectedIDReduction(AnnotDF)

  return AnnotDF

In [13]:
#@title Misc Utilities Modifications

def fig2data(fig):
    """
    @brief Convert a Matplotlib figure to a 4D numpy array with RGBA channels and return it
    @param fig a matplotlib figure
    @return a numpy 3D array of RGBA values
    """
    # Create a new canvas for the figure
    canvas = fig.canvas
    canvas.draw()

    # Get the width and height of the canvas
    w, h = canvas.get_width_height()

    # Get the RGBA buffer from the canvas
    buf = np.frombuffer(canvas.tostring_argb(), dtype=np.uint8)
    buf.shape = (w, h, 4)

    # Roll the ALPHA channel to have it in RGBA mode
    buf = np.roll(buf, 3, axis=2)

    # Destroy the canvas explicitly to release resources
    plt.close(fig)

    return buf

In [18]:
#@title Tracker IO Modifications

import numpy as np
import cv2
import os
import gc
from pycocotools import mask as coco_mask
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd

#from symmetry_tracker.general_functionalities.misc_utilities import fig2data

try:
  from IPython.display import display
  from symmetry_tracker.general_functionalities.misc_utilities import progress
except:
  pass

def SaveTracks(AnnotDF, SavePath):
  """
  Saves the AnnotDF dataframe to a json
  """
  if not SavePath.endswith('.json'):
    raise ValueError("SavePath must have a .json extension")
  AnnotDF.to_json(SavePath, orient='records')

def LoadTracks(LoadPath):
  """
  Loads the AnnotDF dataframe from a json
  """
  if not LoadPath.endswith('.json'):
    raise ValueError("LoadPath must have a .json extension")
  AnnotDF = pd.read_json(LoadPath, orient='records')
  return AnnotDF

def WriteTracks(AnnotDF, SavePath):
  """
  Saves the optimal annotation to a txt (standard format)
  Usually should be performed before interpolating missing cell points, as the interpolated values will be saved as regular (unless this behavior is desired)
  """
  with open(SavePath, "w") as OutFile:
    OutFile.write("POS\tF\tCELLNUM\tX\tY\n")
    for Frame in AnnotDF["Frame"].unique():
      for TrackID in AnnotDF.query("Frame == @Frame")["TrackID"].unique():
        Segmentation = coco_mask.decode(AnnotDF.query("TrackID == @TrackID")["SegmentationRLE"].iloc[0]).astype(np.uint8)
        contours, _  = cv2.findContours(Segmentation ,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        for contour in contours:
          for a in contour:
            l=a[0]
            OutFile.write(str(-1)+"\t"+str(Frame+1)+"\t"+str(TrackID)+"\t"+str(l[0])+"\t"+str(l[1])+"\n")
  print("The track is saved in standard txt format to: ")
  print(SavePath)

def DisplayTracks(VideoPath, AnnotDF, DisplayFrameNumber = True, DisplaySegmentation=True, DisplayTrackIDs = True, DisplayPeriodicity=1, Figsize=(5,5)):
  """
  Displays all cell paths as cell IDs on each frame

  - DisplayFrameNumber: Boolean variable marking whether to display the frame number
  - DisplaySegmentation: Boolean variable marking whether to display the segmentations
                          If True, non-interpolated segments are displayed as "lime", interpolated ones are displayed as "deepskyblue"
  - DisplayTrackIDs: Boolean variable marking whether to display the track IDs
  - DisplayPeriodicity: How frequently should the frames be displayed.
                        Useful for long videos.
                        Minimal possible value=1.
  - Figsize: The shape of the figure in standard matplotlib format
  """
  VideoFrames = sorted(os.listdir(VideoPath))
  for Frame in range(len(VideoFrames)):
    if np.mod(Frame,DisplayPeriodicity)==0:

      Img = cv2.cvtColor(cv2.imread(os.path.join(VideoPath, VideoFrames[Frame])), cv2.COLOR_BGR2RGB)
      fig, (ax1) = plt.subplots(1, 1, figsize=Figsize)
      ax1.imshow(Img, cmap=plt.cm.gray, interpolation='nearest')

      if DisplaySegmentation:
        SegmentsSum = np.zeros(np.shape(Img)[0:2])
        for _, Object in AnnotDF.query("Frame == @Frame").iterrows():
          SegmentsSum += coco_mask.decode(Object["SegmentationRLE"])
        ax1.imshow(SegmentsSum, cmap=plt.cm.hot, vmax=2, alpha=.3, interpolation='bilinear')

      if DisplayFrameNumber:
        ax1.text(3, 20, "Frame "+str(Frame+1), color="deepskyblue")

      if DisplayTrackIDs:
        for _, Object in AnnotDF.query("Frame == @Frame").iterrows():
          color="lime"
          if Object["Interpolated"]:
            color="deepskyblue"
          [cx, cy] = Object["Centroid"]
          ax1.text(cx-5, cy+5, Object["TrackID"], color=color)

      plt.show()

def SaveTracksVideo(VideoPath, AnnotDF, OutputVideoPath,
                    Fps = 10, DisplayPeriodicity=1, StartingFrame=None, EndingFrame=None,
                    DisplayFrameNumber=True, DisplaySegmentation=True, ColoredSegmentation=True, DisplayTrackIDs = True):
  """
  Saves the cell paths in a similar format as DisplayTrack displays them
  Non-interpolated segments are displayed as "lime", interpolated ones are displayed as "deepskyblue"

  - OutputVideoPath: The path to which the video will be saved
  - DisplayFrameNumber: Boolean variable marking whether to display the frame number
  - DisplaySegmentation: Boolean variable marking whether to display the segmentations
  - DisplayTrackIDs: Boolean variable marking whether to display the track IDs
  - Fps: The fps of the saved video
  - Starting Frame: The first frame which should be displayed
                    Can be useful if tracking was only performed after a certain frame
  - Ending Frame: The last frame which should be displayed
                  Can be useful if tracking was only performed after a certain frame
  - DisplayPeriodicity: How frequently should the frames be displayed.
                        Useful for long videos.
                        Minimal possible value=1.
  """
  if OutputVideoPath[-4:]!=".mp4":
    raise Exception("Only video paths with mp4 extension are allowed")

  VideoFrames = sorted(os.listdir(VideoPath))
  if StartingFrame is None:
    StartingFrame = 1
  if EndingFrame is None:
    EndingFrame = len(VideoFrames)

  print("Saving Tracks Video")
  try:
    ProgressBar = display(progress(0, EndingFrame-StartingFrame), display_id=True)
  except:
    pass

  out = cv2.VideoWriter(OutputVideoPath, cv2.VideoWriter_fourcc(*'mp4v'), Fps, (700,700), True)
  for Frame in range(StartingFrame-1,EndingFrame-1):
    if np.mod(Frame,DisplayPeriodicity)==0:

      Img = cv2.cvtColor(cv2.imread(os.path.join(VideoPath, VideoFrames[Frame])), cv2.COLOR_BGR2RGB)
      fig, (ax1) = plt.subplots(1, 1, figsize=(7, 7))
      ax1.imshow(Img, cmap=plt.cm.gray, interpolation='nearest')

      if DisplaySegmentation:

        if ColoredSegmentation:
          SegmentsSum = np.zeros_like(Img)
          cmap = cm.nipy_spectral
          for _, Object in AnnotDF.query("Frame == @Frame").iterrows():
              color = cmap((int(Object["TrackID"])*17)%256)
              mask = coco_mask.decode(Object["SegmentationRLE"])*255
              colored_mask = np.array(np.stack([mask*color[0],mask*color[1],mask*color[2]], axis=2),dtype=np.uint8)
              SegmentsSum += colored_mask
          ax1.imshow(SegmentsSum, vmax=256, alpha=.3, interpolation='bilinear')

        else:
          SegmentsSum = np.zeros(np.shape(Img)[0:2])
          for _, Object in AnnotDF.query("Frame == @Frame").iterrows():
            SegmentsSum += coco_mask.decode(Object["SegmentationRLE"])
          ax1.imshow(SegmentsSum, cmap=plt.cm.hot, vmax=2, alpha=.3, interpolation='bilinear')

      if DisplayFrameNumber:
        ax1.text(1, 10, "Frame "+str(Frame+1), color="deepskyblue", fontsize=10)

      if DisplayTrackIDs:
        for _, Object in AnnotDF.query("Frame == @Frame").iterrows():
          color="lime"
          if Object["Interpolated"]:
            color="deepskyblue"
          [cx, cy] = Object["Centroid"]
          ax1.text(cx, cy, Object["TrackID"], color=color, fontsize=5, ha='center', va='center')

      Img = cv2.cvtColor(fig2data(fig), cv2.COLOR_BGRA2RGB)
      out.write(Img)
      plt.close(fig)

      try:
        ProgressBar.update(progress(Frame, EndingFrame-StartingFrame))
      except:
        pass

  out.release()
  gc.collect()

  try:
    ProgressBar.update(progress(1, 1))
  except:
    pass

In [ ]:
#@title Full Tracking

AnnotPath = SegmentationSavePath

# Performing multiple cell tracking
AnnotDF = SingleVideoObjectTracking(TransformedVideoPath,
                                    TrackingModelPath,
                                    Device,
                                    AnnotPath,
                                    TimeKernelSize=TimeKernelSize,
                                    Color = "RGB",
                                    Marker = "BBOX",
                                    SegmentationConfidence = 0.2,
                                    MinRequiredSimilarity = 0.2,
                                    MaxTimeKernelShift = None)


In [15]:
#@title Interpolation of Missing Objects

# NOTE:
# This step is not based on heuristics, and the results are well determined
# Also, this step is mandatory for successful Inheritance detection
# However it must be performed after Heuristical Post-Processing

AnnotDF = InterpolateMissingObjects(AnnotDF)

Successful interpolation of 1018 object instances


In [16]:
#@title Saving Track results

SaveTracks(AnnotDF,TrackingSavePath)

In [9]:
#@title Loading Track results

AnnotDF = LoadTracks(TrackingSavePath)

In [18]:
#@title Display and Write Tracking results

# Displaying tracking results
#DisplayTracks(TransformedVideoPath, AnnotDF)

# Writing tracking results to standard format txt
WriteTracks(AnnotDF, TrackingWritePath)

The track is saved in standard txt format to: 
./outputs/trackings/600_Tracks.txt


In [19]:
# Saving tracking result to video
SaveTracksVideo(TransformedVideoPath, AnnotDF, TrackingVideoPath, Fps=30, ColoredSegmentation=True, DisplayTrackIDs=False)

Saving Tracks Video


In [1]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
SaveTracks(AnnotDF, "/content/gdrive/MyDrive/General Symmetric Tracking/MOTSynth - CVPR22/codes/refactored predictor/outputs/"+sample_record_name+"_Tracks.json")
SaveTracksVideo(TransformedVideoPath, AnnotDF, "/content/gdrive/MyDrive/General Symmetric Tracking/MOTSynth - CVPR22/codes/refactored predictor/outputs/"+sample_record_name+"_Tracks.mp4")